### Graph Convolutional Network (GCN)

#### 核心思想

- 图卷积层通过**聚合节点的邻居信息**来更新每个节点的特征表示，类似于 CNN 中的卷积操作，但应用于图结构数据

#### 主要变量

1. 初始化参数

- `out_channel`: 每个节点的输出特征维度
- `min_deg/max_deg`: 节点度的最小/最大值，用于处理不同连接度的节点
- `activation_fn`: 非线性激活函数（如 `ReLU`）

2. 输入变量 (`call` 方法)

```python
inputs = [
    atom_features,    # 节点特征矩阵: (n_atoms, n_features)
    deg_slice,        # 度信息切片: (n_degrees, 2), 其中 n_degrees 表示可能的度的数目，2 内部为起始的节点索引和这样的节点有多少个
    # inputs[2] 在代码中未使用，可能是历史遗留
    deg_adj_lists     # 度邻接列表: 每个度对应的邻居索引列表
]
```

3. 核心权重变量

```python
# build 方法中创建的权重
num_deg = 2 * max_deg + (1 - min_deg)               # 权重矩阵数量
W_list: [(input_features, out_channel)] * num_deg   # 每个度一个权重矩阵
b_list: [(out_channel,)] * num_deg                  # 每个度一个偏置向量
```

#### 详细计算流程

1. 权重初始化 (`build` 方法)

```python
num_deg = 2 * max_degree + (1 - min_degree)         # 权重数量
# 例如：min_deg=0, max_deg=10 → num_deg = 21
W_list = [权重矩阵0, 权重矩阵1, ..., 权重矩阵20]
b_list = [偏置向量0, 偏置向量1, ..., 偏置向量20]
```

2. 邻居聚合 (`sum_neigh` 方法)

```python
def sum_neigh(self, atoms, deg_adj_lists):
    # 对每个度的节点，聚合其所有邻居的特征
    for deg in range(1, max_degree + 1):    # 初始值为 1, 因为度为 0 的节点没有邻居
        # 获取该度所有节点的邻居特征
        gathered_atoms = tf.gather(atoms, deg_adj_lists[deg-1])
        # 沿邻居维度求和，得到每个节点的邻居聚合特征
        summed_atoms = tf.reduce_sum(gathered_atoms, 1)
        deg_summed[deg-1] = summed_atoms
```

变量解释：
- `atoms`: 节点特征矩阵，(n_atoms, n_features)
- `deg_adj_lists`: 度邻接列表，`deg_adj_lists[deg-1]` 表示所有度为 `deg` 的节点的邻居索引列表
- `gathered_atoms`: 度为 `deg` 的所有节点的邻居特征，(n_nodes_with_degree, max_degree, n_features)
- `summed_atoms`: 度为 `deg` 的所有节点沿邻居维度求和后的特征结果，(n_nodes_with_degree, n_features)
- `deg_summed`: 将度为 `deg` 的所有节点的邻居聚合特征进行存储

示例：

```python
# 图结构：
#   0 --- 1
#   |     |
#   2 --- 3
#   |
#   4

# 节点度：
# 节点0: 度2 (连接1,2)
# 节点1: 度2 (连接0,3)  
# 节点2: 度2 (连接0,3)
# 节点3: 度2 (连接1,2)
# 节点4: 度1 (连接2)

max_degree = 2

输入数据

# 节点特征
atoms = tf.constant([
    [1.0, 2.0],  # 节点0特征
    [3.0, 4.0],  # 节点1特征
    [5.0, 6.0],  # 节点2特征
    [7.0, 8.0],  # 节点3特征
    [9.0, 1.0]   # 节点4特征
])  # shape: (5, 2)

# 邻接列表（按度组织）
deg_adj_lists = [
    [],           # 度0节点（没有）
    [[2]],        # 度1：节点4的邻居是[2]
    [[1, 2], [0, 3], [0, 3], [1, 2]]  # 度2：节点0,1,2,3的邻居
]

执行过程

# 初始化
deg_summed = [None, None]  # max_degree = 2

# 循环1: deg = 1
deg = 1
gathered_atoms = tf.gather(atoms, [[2]])
# 结果：[[[5.0, 6.0]]]  # 节点4的邻居特征
summed_atoms = tf.reduce_sum(gathered_atoms, 1)
# 结果：[[5.0, 6.0]]  # 节点4的邻居聚合特征
deg_summed[0] = [[5.0, 6.0]]

# 循环2: deg = 2  
deg = 2
gathered_atoms = tf.gather(atoms, [[1, 2], [0, 3], [0, 3], [1, 2]])
# 结果：[
#   [[3.0, 4.0], [5.0, 6.0]],  # 节点0的邻居[1,2]的特征
#   [[1.0, 2.0], [7.0, 8.0]],  # 节点1的邻居[0,3]的特征
#   [[1.0, 2.0], [7.0, 8.0]],  # 节点2的邻居[0,3]的特征
#   [[3.0, 4.0], [5.0, 6.0]]   # 节点3的邻居[1,2]的特征
# ]

summed_atoms = tf.reduce_sum(gathered_atoms, 1)
# 结果：[
#   [8.0, 10.0],  # [3+5, 4+6] 节点0的邻居聚合
#   [8.0, 10.0],  # [1+7, 2+8] 节点1的邻居聚合  
#   [8.0, 10.0],  # [1+7, 2+8] 节点2的邻居聚合
#   [8.0, 10.0]   # [3+5, 4+6] 节点3的邻居聚合
# ]
deg_summed[1] = [[8.0, 10.0], [8.0, 10.0], [8.0, 10.0], [8.0, 10.0]]

最终结果

deg_summed = [
    [[5.0, 6.0]],  # 度1节点的邻居聚合
    [[8.0, 10.0], [8.0, 10.0], [8.0, 10.0], [8.0, 10.0]]  # 度2节点的邻居聚合
]
```

3. 图卷积计算 (call 方法核心)

- 准备阶段

```python
atom_features = inputs[0]           # (n_atoms, n_features)
deg_slice = inputs[1]               # 度信息切片，(n_degrees, 2)
deg_adj_lists = inputs[3:]          # 邻接列表
W = iter(self.W_list)               # 权重迭代器
b = iter(self.b_list)               # 偏置迭代器

# 聚合邻居特征
deg_summed = self.sum_neigh(atom_features, deg_adj_lists)
```

- 按度计算节点

```python
# 按度切分节点特征，内部元素为度相等的节点特征
split_features = tf.split(atom_features, deg_slice[:, 1])

for deg in range(1, max_degree + 1):
    # 获取度为 deg 的节点的邻居聚合特征
    rel_atoms = deg_summed[deg - 1]

    # 获取度为 deg 的节点自身特征
    self_atoms = split_features[deg - min_degree]

    # 分别应用线性变换
    rel_out = tf.matmul(rel_atoms, next(W)) + next(b)       # 邻居部分
    self_out = tf.matmul(self_atoms, next(W)) + next(b)     # 自身部分

    # 相加得到该度节点的输出特征
    out = rel_out + self_out
    new_rel_atoms_collection[deg - min_degree] = out
```

- 处理孤立节点 (度=0)

```python
if min_degree == 0:
    self_atoms = split_features[0]
    # 孤立节点只使用自身特征
    out = tf.matmul(self_atoms, next(W)) + next(b)
    new_rel_atoms_collection[0] = out
```

- 合并所有节点

```python
# 将所有度的节点特征重新拼接
atom_features = tf.concat(axis=0, values=new_rel_atoms_collection)

# 应用激活函数
if activation_fn is not None:
    atom_features = activation_fn(atom_features)
```

#### 数学公式表示

对于度为 d 的节点 i：

$$h_i^{(l+1)} = \sigma(W_{self}^{(d)} \cdot h_i^{(l)} + W_{neigh}^{(d)} \cdot \sum_{j \in N(i)} h_j^{(l)} + b^{(d)})$$

其中：
- $h_i^{(l)}$ 是节点 i 在第 l 层的特征
- $N(i)$ 是节点 i 的邻居集合
- $W_{self}^{(d)}$ 和 $W_{neigh}^{(d)}$ 是度为 d 的节点的权重矩阵
- $\sigma$ 是激活函数

#### 数值示例

假设有一个简单图：
- 节点0（度1）：特征 [1, 2]
- 节点1（度2）：特征 [3, 4]
- 节点2（度1）：特征 [5, 6]
- 连接：0-1-2

```python
# 输入数据
atom_features = [[1, 2], [3, 4], [5, 6]]                # (3, 2)
deg_slice = [[0, 2], [1, 1]]                            # 度0有2个节点，度1有1个节点
deg_adj_lists = [[2], [0, 2]]                           # 度1节点0的邻居是节点2，度2节点1的邻居是0,2

# 权重初始化（假设out_channel=3）
W_list = [W_self_0, W_neigh_0, W_self_1, W_neigh_1, W_self_2, W_neigh_2]
b_list = [b_self_0, b_neigh_0, b_self_1, b_neigh_1, b_self_2, b_neigh_2]

# 计算邻居聚合
deg_summed[0] = [atom_features[2]]                      # 节点0的邻居是节点2
deg_summed[1] = [atom_features[0] + atom_features[2]]   # 节点1的邻居是0和2

# 分别计算各度节点的输出
# 度1节点（节点0,2）：
rel_out_deg1 = deg_summed[0] @ W_neigh_1 + b_neigh_1
self_out_deg1 = atom_features[[0,2]] @ W_self_1 + b_self_1
output_deg1 = rel_out_deg1 + self_out_deg1

# 度2节点（节点1）：
rel_out_deg2 = deg_summed[1] @ W_neigh_2 + b_neigh_2
self_out_deg2 = atom_features[[1]] @ W_self_2 + b_self_2
output_deg2 = rel_out_deg2 + self_out_deg2

# 最终输出合并
final_output = concat([output_deg1, output_deg2])       # 按原始节点顺序
```

#### 关键设计特点

1. **度特化**: 不同度的节点使用不同的权重矩阵，捕获不同连接模式
2. **邻居聚合**: 对邻居特征求和，捕获局部图结构信息
3. **自身+邻居**: 同时考虑节点自身特征和邻居信息
4. **批量处理**: 可以并行处理多个图的节点

这种设计特别适合分子图等具有不规则度分布的图结构数据

---

### LSTMStep

#### 核心概念回顾

- `LSTM` 通过三个门控机制（输入门、遗忘门、输出门）来控制信息的流动，解决传统 RNN 的梯度消失问题

#### 变量详细解释

1. 输入变量

- `x`: 当前时间步的输入向量，形状 (batch_size, input_dim)
- `h_tm1`: 前一时间步的隐藏状态，形状 (batch_size, output_dim)，其中 `output_dim` 包含了四个门的输出维度
- `c_tm1`: 前一时间步的记忆状态（细胞状态），形状 (batch_size, output_dim)

2. 权重矩阵

- `W`: 输入到隐藏的权重矩阵，形状 (input_dim, 4*output_dim)
    - 控制输入 x 对所有门的影响
- `U`: 隐藏到隐藏的权重矩阵，形状 (output_dim, 4*output_dim)
    - 控制前一隐藏状态对所有门的影响

3. 偏置变量

- `b`: 偏置向量，长度 4*output_dim
    - 包含 4 个门的偏置：[b_input, b_forget, b_candidate, b_output]
    - 初始化：[0, 1, 0, 0]（遗忘门偏置为1，倾向于保留信息）

#### 详细计算过程

1. 线性组合计算

```python
z = backend.dot(x, self.W) + backend.dot(h_tm1, self.U) + self.b
```

- `z` 形状：(batch_size, 4*output_dim)
- 这是所有门的原始激活值，尚未经过激活函数

2. 门控值分割

```python
z0 = z[:, :self.output_dim]                     # 输入门原始值
z1 = z[:, self.output_dim:2*self.output_dim]    # 遗忘门原始值  
z2 = z[:, 2*self.output_dim:3*self.output_dim]  # 候选记忆原始值
z3 = z[:, 3*self.output_dim:]                   # 输出门原始值
```

3. 门控激活

```python
i = self.inner_activation_fn(z0)                # 输入门: sigmoid(0,1)
f = self.inner_activation_fn(z1)                # 遗忘门: sigmoid(0,1) 
c = f * c_tm1 + i * self.activation_fn(z2)      # 新记忆状态
o = self.inner_activation_fn(z3)                # 输出门: sigmoid(0,1)
```

4. 最终输出

```python
h = o * self.activation_fn(c)                   # 新隐藏状态
return h, [h, c]                                # 返回隐藏状态和完整状态
```

#### 数学公式详细解析

1. 输入门 (Input Gate)

$$i_t = \sigma(W_i x_t + U_i h_{t-1} + b_i)$$

- 控制多少新信息写入记忆状态
- `i_t` 接近1：大量写入新信息
- `i_t` 接近0：少量写入新信息

2. 遗忘门 (Forget Gate)

$$f_t = \sigma(W_f x_t + U_f h_{t-1} + b_f)$$

- 控制保留多少历史信息
- f_t 接近1：保留更多历史信息
- f_t 接近0：遗忘更多历史信息

3. 候选记忆 (Candidate Memory)

$$\tilde{C}t = \tanh(W_c x_t + U_c h{t-1} + b_c)$$

- 当前时间步的新信息候选
- 通过 tanh 压缩到 [-1, 1] 范围

4. 记忆状态更新

$$C_t = f_t \odot C_{t-1} + i_t \odot \tilde{C}_t$$

- 新记忆状态 = 遗忘的历史 + 新输入的信息
- ⊙ 表示逐元素乘法

5. 输出门 (Output Gate)

$$o_t = \sigma(W_o x_t + U_o h_{t-1} + b_o)$$

- 控制从记忆状态中输出多少信息

6. 隐藏状态输出

$$h_t = o_t \odot \tanh(C_t)$$

- 最终的隐藏状态输出

#### 完整的数值示例

假设参数：

- `batch_size` = 2, `input_dim` = 3, `output_dim` = 4

```python
# 输入
x = [[0.1, 0.2, 0.3], [0.4, 0.5, 0.6]]      # (2,3)
h_tm1 = [[0.1, 0.2, 0.3, 0.4], [0.5, 0.6, 0.7, 0.8]]  # (2,4)  
c_tm1 = [[0.2, 0.3, 0.4, 0.5], [0.6, 0.7, 0.8, 0.9]]  # (2,4)

# 权重矩阵 (随机初始化)
W = [[...], ...]  # (3, 16)
U = [[...], ...]  # (4, 16)  
b = [0,0,0,0, 1,1,1,1, 0,0,0,0, 0,0,0,0]  # (16,)

# 计算过程
z = x @ W + h_tm1 @ U + b  # (2,16)
z0 = z[:, :4]    # 输入门部分 (2,4)
z1 = z[:, 4:8]   # 遗忘门部分 (2,4)
z2 = z[:, 8:12]  # 候选记忆部分 (2,4)  
z3 = z[:, 12:16] # 输出门部分 (2,4)

i = sigmoid(z0)  # 输入门激活值 (2,4)
f = sigmoid(z1)  # 遗忘门激活值 (2,4)
c_candidate = tanh(z2)  # 候选记忆 (2,4)
o = sigmoid(z3)  # 输出门激活值 (2,4)

# 状态更新
c = f * c_tm1 + i * c_candidate  # 新记忆状态 (2,4)
h = o * tanh(c)  # 新隐藏状态 (2,4)
```